# Probabilistic Programming with Haskell

Siddhart Bath, Simeon Carstens, Matthias Meschede

_first published on www.tweag.io/blog_

#### imports

In [ ]:
:e OverloadedStrings
:e ExistentialQuantification

import System.Random.MWC (createSystemRandom)
import qualified Graphics.Vega.VegaLite as VL
import IHaskell.Display.Hvega (vlShow)
import Data.Aeson (ToJSON(toJSON), Value)
import Data.Aeson (encode)
import qualified Data.ByteString.Lazy.Char8 as BL
import Data.Text (Text, pack)

import Control.Monad (liftM2, replicateM, forM, forM_)
import Control.Monad.IO.Class (liftIO)
import Data.List (sort)
import Control.Monad.Bayes.Class
import Control.Monad.Bayes.Sampler
import Control.Monad.Bayes.Traced
import Control.Monad.Bayes.Weighted
import Control.Monad.Bayes.Inference.SMC as SMC
import Control.Monad.Bayes.Inference.RMSMC as RMSMC
import Control.Monad.Bayes.Sequential
import Control.Monad.Bayes.Population
import Control.Monad.Bayes.Traced.Static (Traced)
import Control.Monad.Bayes.Inference.SMC
import Numeric.LinearAlgebra (Matrix, Vector, vector, matrix, dot, (#>), cmap, scalar)

import Numeric.Log
import Control.Monad.Bayes.Class

import Data.List (partition)

#### Plotting

In [ ]:
barPlot :: Text -> VL.VLSpec
barPlot xName = 
    let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Nominal]
            . VL.position VL.Y [VL.PName "binnedData", VL.PAggregate VL.Count, VL.PmType VL.Quantitative, VL.PAxis [VL.AxTitle "count"]]
    in VL.asSpec [VL.mark VL.Bar [VL.MOpacity 1.0, VL.MColor "#a3c6de"], encoding []]

linePlot :: Text -> Text -> VL.VLSpec
linePlot xName yName = 
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Quantitative]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Quantitative]
  in VL.asSpec [VL.mark VL.Line [VL.MColor "blue"], encoding []]

scatterBlue xName yName (xmin, xmax) (ymin, ymax) =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [xmin, xmax]]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [ymin, ymax]]]
  in VL.asSpec [VL.mark VL.Circle [VL.MColor "blue"], encoding []]
  
scatterGreen xName yName (xmin, xmax) (ymin, ymax) =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [xmin, xmax]]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [ymin, ymax]]]
  in VL.asSpec [VL.mark VL.Circle [VL.MColor "green"], encoding []]
  
scatterPlotWithColor :: Text -> Text -> Text -> (Double, Double) -> (Double, Double) -> VL.VLSpec
scatterPlotWithColor xName yName zName (xmin, xmax) (ymin, ymax) =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [xmin, xmax]]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Quantitative, VL.PScale [VL.SDomain $ VL.DNumbers [ymin, ymax]]]
            . VL.color [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]]]
  in VL.asSpec [VL.mark VL.Circle [], encoding []]

density2DPlot :: Text -> Text -> (Double, Double) -> (Double, Double) -> VL.VLSpec
density2DPlot xName yName (xmin, xmax) (ymin, ymax) = 
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PBin [VL.Nice False, VL.Steps [0.05, 0.5, 5.0], VL.Extent xmin xmax], VL.PmType VL.Quantitative]
            . VL.position VL.Y [VL.PName yName, VL.PBin [VL.Nice False, VL.Steps [0.05, 0.5, 5.0], VL.Extent ymin ymax], VL.PmType VL.Quantitative]
            . VL.color [ VL.MAggregate VL.Count, VL.MName "col", VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]]]
  in VL.asSpec [VL.mark VL.Rect [], encoding []]

imagePlot :: Text -> Text -> Text -> VL.VLSpec
imagePlot xName yName zName =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Nominal, VL.PAxis [VL.AxGridOpacity 0.1]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Nominal, VL.PSort [VL.Descending], VL.PAxis [VL.AxGridOpacity 0.1]]
            . VL.fill [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]]]
            . VL.stroke [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]],
                          VL.MLegend [VL.LType VL.GradientLegend]]
  in VL.asSpec [VL.mark VL.Rect [], encoding []]
  
imageFacetPlot :: Text -> Text -> Text -> VL.VLSpec
imageFacetPlot xName yName zName =
  let encoding = VL.encoding
            . VL.position VL.X [VL.PName xName, VL.PmType VL.Ordinal, VL.PAxis [VL.AxGrid False]]
            . VL.position VL.Y [VL.PName yName, VL.PmType VL.Ordinal, VL.PSort [VL.Descending], VL.PAxis [VL.AxGrid False]]
            . VL.fill [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]], VL.MLegend [VL.LOrient VL.LOBottom]]
            . VL.stroke [ VL.MName zName, VL.MmType VL.Quantitative, VL.MScale [VL.SScheme "blues" [0.0, 1.0]],
                          VL.MLegend [VL.LOrient VL.LOBottom, VL.LDirection VL.Horizontal, VL.LType VL.GradientLegend]]
  in VL.asSpec [VL.mark VL.Rect [], encoding [], VL.width 200,  VL.height 100]

data SpecGrid = H [[VL.VLSpec]] | V [[VL.VLSpec]] | L [VL.VLSpec] | S VL.VLSpec | F (Text, Int, VL.VLSpec)

data InputData = Cols [(Text, VL.DataValues)]
               | File FilePath

plot :: (Double, Double) -> SpecGrid -> InputData -> VL.VegaLite
plot (figw,figh) specGrid dataPoints =
    let description = VL.description "Plot"
        dat' = case dataPoints of
            Cols cols -> foldl (.) (VL.dataFromColumns []) (map (uncurry VL.dataColumn) cols) []
            File fp -> VL.dataFromSource (pack fp) []
        configure = VL.configure
            . VL.configuration (VL.Axis
                                        [ VL.Domain False,
                                          VL.LabelColor "#7F7F7F",
                                          VL.LabelPadding 4,
                                          VL.TickColor "#7F7F7F",
                                          VL.TickSize 5.67,
                                          VL.Grid True,
                                          VL.GridColor "#FFFFFF"
                                          ])
        spec = case specGrid of
            S s -> VL.layer [s]
            L ls -> VL.layer ls
            H lss -> VL.hConcat (map (VL.asSpec . (:[]) . VL.layer) lss)
            V lss -> VL.vConcat (map (VL.asSpec . (:[]) . VL.layer) lss)
            F (_, _, s) -> VL.specification s
        facet = case specGrid of
            F (field, nColumns, _) -> [VL.columns $ fromIntegral nColumns, VL.facetFlow [VL.FName field, VL.FmType VL.Nominal]]
            _   -> [VL.width figw,  VL.height figh]
    in VL.toVegaLite $ [VL.background "#f9f9f9", configure [], description, dat', spec] ++ facet

### Introduction

In this blog post series, we're going to lead you through Bayesian modeling in Haskell with the [`monad-bayes`](https://github.com/adscib/monad-bayes/tree/647ba7cb5a98ae028600f3d828828616891b40fb) library.
We start this series gradually with some simple binary models, move next to linear regression, and finish by building a simple neural network that we "train" with a Metropolis-Hastings sampler.
You don't need any prior knowledge of Bayesian modeling to understand and learn from these posts—and we keep the code simple and understandable for Haskell newcomers.

Want to make this post interactive? Try our [notebook version](https://gist.github.com/MMesch/dee0659e480b572d0b36088a2b08cc57). It includes a Nix shell, the required imports, and some helper routines for plotting. Let's start modeling! 

## Part 1: First Steps

### Sampling

In this first part of the series, we introduce two fundamental concepts of `monad-bayes`: `sampling` and `scoring`.
We examine them based on one of the simplest probabilistic models that we can think of—a model that represents a `True` or `False` choice.
You can use it, for example, to describe the answer to a question such as "Did it rain yesterday?".

The model is parameterized by a boolean `b`, and in this simple case, `b` is also directly the model output.
Without additional information, we assign equal probabilities `0.5` to each value that `b` can take (50% `True`, 50% `False`).
In other words, we get the model parameter `b` from a discrete uniform [prior](https://en.wikipedia.org/wiki/Prior_probability) distribution.

Let's see how the model looks like in the`monad-bayes`library:

In [ ]:
model1 :: MonadSample m => m Bool
model1 = do
    b <- uniformD [False, True]
    return b

In `monad-bayes` a model is expressed through the typeclass `MonadSample`.
The `MonadSample` typeclass provides a function `random` to our `model1` that returns a random sample from it.
The type of the sample itself is set to `Bool` in our case.

We define our model by binding together other basic `MonadSample` models.
In this case, for example, we build our model from the `uniformD` distribution that is provided by `monad-bayes`
The model, that is the chain of actions that are bound together in _MonadSample_, is not executed until we start sampling from the model.
Once we sample, the resulting chain of actions that is executed is simple:
draw `b` from a discrete uniform distribution (_uniformD_) and then return its value.

Sampling can be executed with:

In [ ]:
sampleIOfixed model1

We can get a list of samples with Haskell's `replicateM` function:

In [ ]:
nsamples = 1000
samples <- sampleIOfixed $ replicateM nsamples model1

Then plot the result afterwards with [Vega-Lite](https://github.com/vega/vega-lite#readme) (module `VL`).
You can find our custom plotting functions for Vega-Lite in the notebook.

In [ ]:
vlShow $ plot (200, 100) (L [barPlot "b"]) (Cols [("b", VL.Booleans samples)])

So far, so good: we now have a model that represents a distribution of `True/False` values and we can draw samples from it.
But how can we include observations into this model?

## Scoring

Consider again `model1` as the answer to "Did it rain yesterday?".
What if we found out "Yes, it did rain yesterday!"?
To include this new knowledge, we need to update the distribution of the model parameter `b`.

In `monad-bayes` the function `score` is responsible for making samples more or less likely—by a factor.
Don't worry if you are mystified by this explanation, we'll explain more in a moment.
But first check out `model2` that uses `score` to include the observation:

In [ ]:
model2 :: MonadInfer m => m Bool
model2 = do
    b <- uniformD [False, True]
    score (if b then 1.0 else 0.0)
    return b

Notice the new typeclass `MonadInfer` that allows us to use `score` in addition to sampling.
Instead of a single operation, the model is now a chain of actions: _sample_, _score_, …

Here's a naive idea of how this could work:
assume that the representation of the probability distribution of `b` was a list of tuples `[(0.5, True), (0.5, False)]`.
We would multiply this distribution with the distribution of my observation "Yes, it did rain yesterday!", that is with `[(1, True), (0, False)]`.
Then we would normalize the updated probabilities such that they sum to one, and the job is done.

So are we secretly tracking the probability of all samples—the full distribution—at every step (in the Haskell world a.k.a. some variant of the [_Dist_ monad](http://www.randomhacks.net/files/build-your-own-probability-monads.pdf))?
Granted, in the case of a `True`/`False` question this might be a good approach—_but this is not what is happening here for very good reasons_:

To update a sample's probability as described above we need to track all samples with their probabilities and run global normalization operations on them.
We are essentially running computations with full distributions over all possible values.
This quickly becomes intractable because a model can basically be any Haskell function with lots and lots of possible outcomes.

`monad-bayes` and similar probabilistic frameworks use an elegant approach that do what we want _without dragging around full distributions_.
The trick behind is to approximate the outcome distribution by drawing successive samples from it.
It turns out that it is enough to know the _relative probability_ of two samples at a time to do this.
The relative probability of the two samples is independent of the probability of other samples—computations on full distributions are thus reduced to computations on samples.
With appropriate sampling algorithms ([MCMC](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo)) we can approximate the outcome distribution despite this limited knowledge.
It is difficult to overstate the implications:
with MCMC we can address all kinds of sampling related problems in a probabilistic manner that would be completely inaccessible otherwise.

Let's get back to the `score` function that modifies the probability to pull a certain sample from the distribution.
We now understand that it multiplies the _relative probability_ to observe a sample compared to any other with a factor.
This means that the sample's probability is left untouched if this factor is `1`.
If this factor is `10`, the sample's relative probability is increased ten fold with respect to any other sample.
If this factor is `0`, the sample's probability is set to `0`.

Here is `model2` expressed in words: (a) draw a sample from `[True, False]` with equal probability,
and (b) multiply the relative probability of a sample with value `True` with `1` and of a sample with value `False` with `0`.

An appropriate sampler can trace and accumulate the score factor of a sample to compare with the score factor of other samples.
The accumulated score factors give us access to the relative probability of the two samples, and then we can use MCMC to start sampling.
We won't go into detail here about how tracing and accumulating works or which two samples we are actually comparing.
`monad-bayes` provides a few samplers that can go through this process in different ways.
We include here the `prior` and the `mh` ([Metropolis-Hastings](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm) sampler) functions before our well known `sampleIOfixed` function to use an MCMC sampler.
Hopefully, the general idea became clear enough here. We'll provide more of the details later in this series.

The final result looks like this:

In [ ]:
nsamples = 1000
samples <- sampleIOfixed $ prior $ mh nsamples model2
vlShow $ plot (200, 100) (L [barPlot "b"]) (Cols [("b", VL.Booleans samples)])

Voilà, we wrote down a model (answer to "Did it rain yesterday?") with an uninformed (uniform) prior, and updated it based on the observation "It rained yesterday!".
The distribution of parameter `b` after scoring—its posterior distribution—has probability `1` for True and probability `0` for False.
The operations that we needed to figure out the posterior distribution were _random_ and _score_.

### Multiple parameters

Let's move onwards to more complex models:
What if we considered a model with two parameters?
Both parameters are drawn independently from uniform _continuous_ distributions between `-1` and `1`.
Again, we want to score this model.
This time, we use the function `condition` that is a short form for scoring with `1` or `0` based on a condition.
The new model becomes:

In [ ]:
model3 :: MonadInfer m => m (Double, Double)
model3 = do
    b <- uniform (-1) 1
    m <- uniform (-1) 1
    condition $ (b-m) > 0
    return (b, m)

The principal approach is the same: pick sample `b`, pick sample `m`, modify the joint sample probability based on a condition, and return the values of both samples in a tuple.
If we run this through the Metropolis-Hastings sampler, we get:

In [ ]:
nsamples = 5000
modelsamples <- sampleIOfixed $ prior $ mh nsamples model3
(xValues, yValues) = unzip modelsamples
vlShow $ plot (600, 300)
              (L [density2DPlot "b" "m" (-1.1,1.1) (-1.1,1.1)])
              (Cols [("b", VL.Numbers xValues), ("m", VL.Numbers yValues)])

The resulting distribution in the plot above is `0` where `b<m`, and approximately uniform when `b>m`, as we'd expect.
You might spot the initial state of the Markov chain as a faint rectangle in the `b<m` region.

How about multiple conditions?
Remember, we can freely bind operations together so it shouldn't be a problem.
This model chains two sampling and two condition operations:

In [ ]:
model4 :: MonadInfer m => m (Double, Double)
model4 = do
    b <- uniform (-1) 1
    m <- uniform (-1) 1
    condition $ (b-m) > 0
    condition $ (b+m) > 0
    return (b, m)

And it produces the expected result:

In [ ]:
nsamples = 5000
modelsamples <- sampleIOfixed $ prior $ mh nsamples model4
(xValues, yValues) = unzip modelsamples
vlShow $ plot (600, 300)
              (L [density2DPlot "b" "m" (-1.1,1.1) (-1.1,1.1)])
              (Cols [("b", VL.Numbers xValues), ("m", VL.Numbers yValues)])

### Conclusions

We have seen how we can build models and examine related probability distributions by drawing samples and modifying their relative probabilities with the `score` function.
The MCMC approach taken by`monad-bayes`and similar frameworks avoids computations with full distributions, and works with individual samples to enormously simplify life.
`monad-bayes`can, therefore, be used to approximate large and complex distributions—something that quickly comes in handy.
We can use`monad-bayes`to approximate the distribution of the return values of basically any Haskell function for a given input distribution.
This could even be the return values of entire programs.
Even better—we can use `score` to infer the input distribution if we have a way of scoring samples based on observations.

We hope you enjoyed this first post in our _Probabilistic Programming with monad&#8209;bayes Series_ and learned lots! Now, you're ready to build more general statistical models using these building blocks, and proceed to linear regression in our next post. We hope you join us!

part 2 (link to come)

## Notes

We use [this GitHub version](https://github.com/adscib/monad-bayes/tree/647ba7cb5a98ae028600f3d828828616891b40fb) of`monad-bayes`in our posts and notebooks since it's neither on Hackage nor Stackage right now. Here are two original articles you may want to check out:

* [Practical Probabilistic Programming with Monads](http://mlg.eng.cam.ac.uk/pub/pdf/SciGhaGor15.pdf)
* [Functional Programming for Modular Bayesian Inference](http://denotational.co.uk/publications/scibior-kammar-ghahramani-funcitonal-programming-for-modular-bayesian-inference.pdf)

## Part 2: Linear Regression

This post is a continuation of Tweag's _**Probabilistic Programming with monad‑bayes Series**_. You can find {Part 1 here](https://www.tweag.io/posts/2019-09-20-monad-bayes-1.html).
Want to make this post interactive? Try our [notebook version](https://github.com/tweag/blog-resources/tree/master/monad-bayes-series). It includes a Nix shell, the required imports, and some helper routines for plotting. Let's start modeling!

### Modeling

The lawn is wet, did it rain?
The ground moves, is there an earthquake?
Prices rise, are we in an economic crisis?
Wiggly lines appear on the screen, did a black hole perturb gravity on Earth?
Patterns emerge in software [dependency graphs](https://www.tweag.io/posts/2019-02-06-mapping-open-source.html) and [source code](https://www.tweag.io/posts/2019-08-01-codestatistics-umap.html), is this the outcome of a particular development style?
Evidence is found in a legal discovery procedure, is anyone guilty of wrongdoing?

We continuously make observations similar to "the lawn is wet".
Such data touches our senses either directly, or indirectly through a measurement device.
It is visible, concrete, and therefore unquestioned here—contrary to the invisible and abstract processes that might have generated it.

Neither can every process generate this data—
earthquakes don't (usually) wet the lawn, black holes don't (yet) influence prices, and legal malpractice doesn't (often) bring about specific source code patterns—
nor is there a unique process that can generate it:
earthquakes and black holes can both move the ground; sprinklers and rain can wet the lawn equally well.
Hence, a process implies data, but data doesn't imply a process.

The conclusion is that we can't conclude that a single process is behind the data, but processes aren't indistinguishable either.
We can _compare_ them by assessing how likely it is that they generate the data.
Bayesian inference and the laws of probability tell us _how_ to make this comparison rationally.

Conceptually it is simple:
Define a set of processes that we want to compare and prior beliefs in each of them (the _prior distribution_). 
For each of these processes, describe and compute the probability that it generates the observed data (the _likelihood_).
Score each process - that is multiply the prior belief in a process with the likelihood.
Renormalize and examine the result, the posterior beliefs in each model (the _posterior distribution_).

In this blog post, we walk through this process step by step with [`monad-bayes`](https://github.com/adscib/monad-bayes#readme).
First, we set up a _statistical model_ that describes an ensemble of data generation processes.
Then, we generate synthetic data with one specific process in this ensemble.
From this data, we try to infer which process we have used from the ones that our statistical model describes.

### Model Setup

The data that we consider are two-dimensional points (x, y).
These points could be observations of time and velocity, location and temperature, velocity and acceleration, or any other two continuous variables that we observe.
Here is how we can describe such data with Haskell:

In [ ]:
data Data
  = Data
      { xValue :: Double,
        yValue :: Double
      }
  deriving (Eq, Show)

The model that we consider generates data along lines with some Gaussian noise around.
This means that the x and y values of our data points cannot take any value, but that `y` can be calculated from `x` with the equation `m*x + b + n`, with parameters `m,b` that are the same for all points and a random, normally-distributed deviation `n` that is independently drawn for each point.
In Haskell, the model, i.e., the lines that it describes, can thus be characterized with:

In [ ]:
data Params
  = Params
      { slope :: Double,
        intercept :: Double,
        noiseStd :: Double
      }
  deriving (Eq, Show)

The likelihood, the heart of our model, corresponds to this succinct Haskell function:

In [ ]:
likelihood :: Params -> Data -> Log Double
likelihood (Params m b nStd) (Data x y) = normalPdf (m*x + b) nStd y

It describes the probability (`Log Double`) to observe some data (`Data`) given some model parameters (`Params`).
In our case, it's just a Gaussian distribution (`normalPdf`), centered at `m*x + b` for a given `x` and standard deviation `nStd`.
That's it—the model definition is unspectacular and short.

### The Model As A Family Of Distributions

For a given value of `Params`, `likelihood` provides us with the (log-)probability for a `Data` point.
In other words, it describes a probability distribution.
We can pick data samples from such a _sampling distribution_—modeling a real-world data generating process.
For instance, let's choose the following parameters:

In [ ]:
 params0 = Params {slope=2.3, intercept=(-1.2), noiseStd=2.0} 

The associated sampling distribution can then be obtained with:

In [ ]:
samplingDistribution' :: Data -> Double
samplingDistribution' = exp . ln . likelihood params0

The extra `exp . ln` snippet here is required to extract a `Double` from a `Log Double`, a data type that internally stores the logarithm of a number instead of the actual value.
Here is a plot that shows how this 2D sampling distribution looks like:

In [ ]:
points1 = [(x, y, samplingDistribution' (Data x y)) | x<-[-10..10], y<-[-10..10]]

vlShow $ plot (600, 300)
              (L [imagePlot "x" "y" "z"])
              (Cols [("x", VL.Numbers ((\(x, _, _)->x) <$> points1)),
                     ("y", VL.Numbers ((\(_, x, _)->x) <$> points1)),
                     ("z", VL.Numbers ((\(_, _, x)->x) <$> points1))])
-- this plot needs a newer version of jupyterlab and Vega. To see
-- the correct version, click on the 3 points in the upper right
-- corner and open it in the vega editor

You can think about it as a line with some noise around, or as a Gaussian distribution with fixed standard deviation in `y` direction and a mean that increases linearly with `x`, which is closer to the definition.

Our statistical model doesn't just describe a single distribution but a different one for each value of `Params`.
A bunch of `Params` therefore corresponds to a bunch of sampling distributions.
Or, expressed more confusingly, a distribution of `Params` corresponds to a distribution of sampling distributions.
Take, for example, a distribution of `Params` that we call `priorParams` where slope, intercept and noise standard deviation are drawn from uniform distributions:

In [ ]:
priorParams :: MonadSample m => m Params
priorParams = do
  intercept <- uniform (-5) 5
  slope <- uniform (-5) 5
  noise <- uniform 1 3
  return $ Params slope intercept noise

We can draw a few parameters from this model parameter distribution:

In [ ]:
mkSampler = replicateM 9
params <- sampleIOfixed $ mkSampler priorParams

And compute and plot the corresponding sampling distributions for each parameter sample in the domain `x=[-10, 10]` and `y=[-10, 10]`:

In [ ]:
points2 =
  [ [ (iparam, x, y, prob (Data x y))
      | x <- [-10, -9.5 .. 10],
        y <- [-10, -9.5 .. 10],
        let prob = exp . ln . likelihood param
    ]
    | (iparam, param) <- zip [0 ..] params
  ]

is = mconcat $ (map . map) (\(i, _, _, _) -> i) points2
xs = mconcat $ (map . map) (\(_, x, _, _) -> x) points2
ys = mconcat $ (map . map) (\(_, _, y, _) -> y) points2
zs = mconcat $ (map . map) (\(_, _, _, z) -> z) points2

vlShow $ plot (600, 300)
              (F ("iParam", 3, imageFacetPlot "x" "y" "likelihood"))
              (Cols [("iParam", VL.Numbers is),
                     ("x", VL.Numbers xs),
                     ("y", VL.Numbers ys),
                     ("likelihood", VL.Numbers zs)])
-- this plot needs a newer version of jupyterlab and Vega. To see
-- the correct version, click on the 3 points in the upper right
-- corner and open it in the vega editor

Each distribution varies in terms of its slope, intercept and standard deviation.
Note that the maximum of a distribution is lower if it has a higher standard deviation, which is due to the normalized `normalPdf` function we used.
However, we didn't properly normalize these distributions within the limited 2D domain but that's not important for what is coming.
The important point to understand is that our `likelihood` model describes a family of distributions which are parametrized through `Params`.

### Generating Data - MCMC

Now let's pick one of those distributions, the one with `Params {slope=2.3, intercept=(-1.2), noiseStd=2.0}` that we showed initially, and draw some synthetic data samples from it.

To sample with `monad-bayes`, we express `likelihood` directly as a distribution.
This means, `likelihood :: Params -> Data -> Log Double` becomes `likelihoodDist :: Params -> m Data`.
Here, `m` implements the `MonadInfer` typeclass that represents a distribution from which we can sample.
It thus somehow deals with the `Log Double` probability behind the scenes:

In [ ]:
likelihoodDist :: MonadInfer m => Params -> m Data
likelihoodDist params = do
  x <- uniform (-10) 10
  y <- uniform (-10) 10
  score $ likelihood params (Data x y)
  return $ Data x y

As we did in our [previous blog post](https://www.tweag.io/posts/2019-09-20-monad-bayes-1.html), we can sample from this distribution with [MCMC](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo):

In [ ]:
mkSampler = prior . mh 300
pointsMCMC <- sampleIOfixed $ mkSampler $ likelihoodDist params0

vlShow $ plot (600, 300)
              (L [scatterBlue "x" "y" (-10, 10) (-10, 10)])
              (Cols [("x", VL.Numbers (xValue <$> pointsMCMC)), ("y", VL.Numbers (yValue <$> pointsMCMC))])

Did this work?
Well, we got some samples, and they are roughly distributed according to the desired distribution that we showed above.
But, there is a problem:

First, there is an improbable outlier sample all the way on the left.
We already blamed this on the initial state of the [Markov chain](https://en.wikipedia.org/wiki/Markov_chain_Monte_Carlo) [previously](https://www.tweag.io/posts/2019-09-20-monad-bayes-1.html).
We will show you how to skip this initial state later in this post.

Then, the samples are not distributed as we would expect.
They don't cover the full x-range, which we could fix by pulling more samples with MCMC—but still.
And there is a deeper issue, the samples seem to be aligned along the y-axis (and the x-axis if you look closely).
The reason is obvious, each sample in the Markov chain (except for the initial one) depends on the previous sample.
The distribution of these samples would eventually converge to the one described by `likelihood`, but they are _not independent_.
So, we need a different technique here to draw samples that _are independent_.

### Generating Data - Rejection Sampling

Many possible ways exist to draw independent samples from a given distribution, but few approaches are accurate and efficient in any situation.
Here we choose an inefficient but quite general approach called [rejection sampling](https://en.wikipedia.org/wiki/Rejection_sampling).
It works like this:
We first get a bunch of independent uniform 2D data points via `monad-bayes`.

In [ ]:
uniform2D :: MonadSample m => m Data
uniform2D = do
  x <- uniform (-10) 10
  y <- uniform (-10) 10
  return $ Data x y

We then draw a list of `2000` points from this distribution with the help of `replicateM` and `sampleIOfixed` :

In [ ]:
mkSampler = replicateM 2000 
uniformSamples <- sampleIOfixed $ mkSampler uniform2D

These samples aren't distributed according to the desired probability, our sampling distribution, but it is easy to compute it for each of these samples:

In [ ]:
desiredProb = map samplingDistribution' uniformSamples

Now we can reject unlikely samples and accept the likely ones to get what we want.
To decide which ones to accept, we draw a uniform random number for each sample.
A sample is only accepted if this number is higher than the desired probability:

In [ ]:
uniform0 <- sampleIOfixed $ mkSampler $ uniform 0 (maximum desiredProb)
points3 = [p | (p, u, l)<-zip3 uniformSamples uniform0 desiredProb, u<l]
length points3

The rejection procedure removes ~90% of all uniformly distributed points and reduces the initial number of `2000` points to only `190`.
But the probability to accept is proportional to the desired probability.
The remaining points are therefore independent and distributed according to our desired sampling distribution, as this plot demonstrates:

In [ ]:
vlShow $ plot (600, 300)
              (L [scatterBlue "x" "y" (-10, 10) (-10, 10)])
              (Cols [("x", VL.Numbers (xValue <$> points3)), ("y", VL.Numbers (yValue <$> points3))])

Rejection sampling is often not very efficient, but it is simple and straight forward.
By design, most rejected samples are in low probability regions, and it is thus crucial to choose an initial distribution that is concentrated around high probability regions—if we know where they are.
Long story short, we now have data samples that correspond to the sampling distribution identified by `Params {slope=2.3, intercept=(-1.2), noiseStd=2.0}`.

### Linear Regression - Inferring Slope and Intercept

Can we find the parameters that we have used from the data points only?
In other words, can we find the _specific_ distribution that generated the data points from the _family_ of distributions that our model describes?
Well, at least we can assess how probable it is that a distribution in the model generates the data - the likelihood.
With this likelihood, we can score the distributions from the prior distribution of parameters `priorParams` that we have defined before.
Now pay attention, here is the inference recipe, in English and in Haskell:

_pull out a model parameter sample from the prior distribution and score it with the likelihood to observe all data points_:

In [ ]:
postParams :: MonadInfer m => m Params -> [Data] -> m Params
postParams pr obs = do
  param <- pr
  forM_ obs (\point -> score (likelihood param point))
  return param

Remember, the `score` function multiplies the relative probability of a model parameter sample with a factor:
Our belief in model parameters that correspond to distributions that _likely_ generate what we observe are multiplied with a _high_ number.
Our belief in model parameters that correspond to distributions that are _unlikely_ to generate what we observe are multiplied with a _low_ number.
We thus update the prior probabilities of model parameters and get a new posterior distribution of model parameters.

But now the question is again how we can sample from this posterior distribution.
The sampler that we use to handle the `score` operation, that is updating relative probabilities of samples in a distribution, is [Metropolis-Hastings](https://en.wikipedia.org/wiki/Metropolis%E2%80%93Hastings_algorithm):

In [ ]:
mkSampler = prior . mh 1000
modelsamples <- sampleIOfixed $ mkSampler $ postParams priorParams points3

Here is how the posterior distribution looks like:

In [ ]:
stable = take 800 modelsamples
vlShow $ plot (600, 300)
              (L [density2DPlot "intercept" "slope" (-5, 5) (-5, 5)])
              (Cols [("intercept", VL.Numbers (intercept <$> stable)), ("slope", VL.Numbers (slope <$> stable))])

This posterior distribution over model parameters describes how likely each sampling distribution in the model generates the observed data.
It peaks at an intercept of `-1.5` - `-1` (the actual value was `-1.2`) and a slope of `1.5`-`2.5` (the actual value was `2.3`).
It seems to work, although we should have probably sampled a bit more to make this more accurate.

You might have noticed that we only use 800/1000 samples here (`take 800` in the code snippet above).
What is the reason? 
If you remember, we saw some random initial samples of the Markov chain in the posterior model parameter distribution in our [last post](https://www.tweag.io/posts/2019-09-20-monad-bayes-1.html).
This was because the chain starts at a random position and needs some steps to reach the stable equilibrium that describes the posterior distribution.
We therefore skip the initial part of it.
Why do we use `take` rather than `drop` for this?
Samples are by default appended at the beginning of the list.
Therefore, `take 800` means that we use the `800` latest samples of the chain and drop the `200` initial ones.
Want to know why we skipped 200 and not 20 points? Check out the Appendix for more details!

### Simulating Data

The posterior distribution of the model parameters tells us which distributions likely generate the observed data.
What data would we generate from the sampling distributions defined by this posterior distribution?

Let's write a new sampler for a _predictive distribution_:

In [ ]:
predDist :: MonadInfer m => m Params -> m Data
predDist paramDist = do
  params <- paramDist
  point <- likelihoodDist params
  return point

We draw samples from it with Metropolis-Hastings:

In [ ]:
mkSampler = prior . mh 40000 
pts <- sampleIOfixed $  mkSampler $ predDist $ postParams priorParams points3
predPoints = take (length pts - 100) pts

The data looks like this:

In [ ]:
vlShow $ plot (600, 300)
              (L [density2DPlot "x2" "y2" (-10, 10) (-10, 10), scatterGreen "x" "y" (-10, 10) (-10, 10)])
              (Cols [("x", VL.Numbers (xValue <$> points3)), ("y", VL.Numbers (yValue <$> points3)),
                     ("x2", VL.Numbers (xValue <$> predPoints)), ("y2", VL.Numbers (yValue <$> predPoints))])

This looks quite OK; the simulated distribution (blue) looks similar to the original data (green).
Although we have drawn 40000 samples with MCMC, you see that the distribution is still not as smooth and nice as we would expect.
Of course, we could sample more, but we really should think about our sampling method and see how we adapt it to the problem and make it more efficient.
But, that's not the aim of this post, so we'll postpone that discussion to later in our series.

Note that `predDist` looks _very_ similar to `likelihoodDist`.
The fundamental difference is that the former takes a distribution, in this case the _posterior predictive_ distribution `postParams priorParams points :: m Params` as input, whereas the latter accepts a single value, in this case `params :: Params`.
The resulting distributions, that we are comparing in the plot above, `likelihoodDist params` and `predDist (postParams priorParams points)` look similar—and it is fairly obvious why:
The posterior predictive distribution peaks at values close to `params` and is zero elsewhere.
Why does it peak at these values?
Well, we gave a high score to those model parameters that are likely to produce the same data as `params0`.
This explanation becomes somewhat circular, but it illustrates again _why_ the predicted points look like the ones that we used for fitting by design.

### Conclusions

We went step by step through the modeling process with `monad-bayes` and a linear regression example.
We learned that the model setup takes only a few lines of code, that is the implementation of `Data`, `Params`, and `likelihood` plus the definition of `prior` and `posterior`.
With these initial definitions, the composability of Haskell functions allows us to rapidly get new distributions like `samplingDistribution` or a `predDist`.
Involved and with big implications is the sampling process, the choice of the sampling technique, and how it is adapted to the current situation.
To sample, we used Metropolis-Hastings and rejection sampling but many other sampling strategies exist.

We hope you enjoyed this second post in our Probabilistic Programming with monad‑bayes Series and learned lots! A central aspect of `monad-bayes` is to enable the modeller to use and compose different sampling techniques for their models. Now, you're ready to start getting into such questions in our next post. We hope you join us!

### Appendix - Stability of the Monte Carlo chain

The Markov chain, that we used to sample in this blog post, is a stochastic process that cycles through possible models.
Models that are more likely are hit more often than models that are less likely, and over time we thus recover the model distribution that we are interested in.
However, typically it needs a while to reach the unique stationary distribution that is equal to the model distribution distribution.
A simple way to assess when it becomes stationary is to look at the likelihood that it reaches:

In [ ]:
likelihoods = [Numeric.Log.sum (map prob points3) | param <- modelsamples, let prob = likelihood param]
nsamples = length likelihoods

vlShow $ plot (600, 300)
              (L [linePlot "imodel" "likelihood"])
              (Cols [("imodel", VL.Numbers $ fromIntegral <$> [nsamples, nsamples-1..0]), ("likelihood", VL.Numbers (ln <$> likelihoods))])

Our MC chain is non-stationary in the beginning at `imodel` values up to 100, and then reaches stationary behaviour.

### Notes

We use [this GitHub version](https://github.com/adscib/monad-bayes/tree/f55d9fa9d24d169d53bb03598306ee8c46b5fc11) of`monad-bayes`in our posts and notebooks since it's neither on Hackage nor Stackage right now.